<a href="https://colab.research.google.com/github/Samar-mami18/SimpleAgent/blob/main/FirstAgentThabet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
transformers_version = "v4.29.0"

print(f"Setting up everything with transformers version {transformers_version}")

!pip install huggingface_hub>=0.14.1 git+https://github.com/huggingface/transformers@$transformers_version -q diffusers accelerate datasets torch soundfile sentencepiece opencv-python openai
!pip install beautifulsoup4
!pip install requests

In [0]:
from huggingface_hub import notebook_login
notebook_login()

In [0]:
from transformers import Tool
import requests
from bs4 import BeautifulSoup

class TechTracker(Tool):
  name = "Hacker_Scraper"
  description = ("This is a tool that scraps the Hacker News website. It takes ino input and returns a list of tuples named `titles_links` containing the top scored websites titles and links.")
  inputs=[]
  outputs=["text"]

  def __call__(self):
    res = requests.get("https://news.ycombinator.com/news")
    soup = BeautifulSoup(res.text, "html.parser")

    titlelines = soup.select(".titleline")
    scores = soup.select(".score")

    scores = [int(score.contents[0].split(' ')[0]) for score in scores]
    titles_links = [(titlelines[i].contents[0].contents[0], titlelines[i].contents[0].get('href')) for i,score in enumerate(scores) if score>100]
    return titles_links

In [0]:
hacker_scraper_tool = TechTracker()

In [0]:
from transformers.tools import HfAgent

agent = HfAgent("https://api-inference.huggingface.co/models/bigcode/starcoder", additional_tools=[hacker_scraper_tool])

In [0]:
agent.toolbox

In [0]:
agent.run("scrap the Hacker News website")

In [0]:
texts = [BeautifulSoup(requests.get(title_link[1]).text, "html.parser").get_text() for title_link in titles_links if BeautifulSoup(requests.get(title_link[1]).text, "html.parser").get_text().find("JavaScript is not available")==-1]
